In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd

# Pour un style visuel plus agréable
plt.style.use('ggplot')

###############################################################################
#                          PARAMÈTRES GLOBAUX
###############################################################################
Fe    = 44100               # Fréquence d'échantillonnage
baud  = 200                 # Débit binaire en bit/s
Ns    = int(Fe / baud)      # Nombre d'échantillons par bit (~220)
Fp_ASK = 2000               # Porteuse ASK
fp1    = 200                # Porteuse FSK (bit = 1)
fp2    = 3000               # Porteuse FSK (bit = 0)

###############################################################################
#             FONCTION DE LECTURE AUDIO SÉCURISÉE (safe_play)
###############################################################################
def safe_play(signal, samplerate):
    """
    Joue le signal audio en toute sécurité.
    Enveloppe sd.play() et sd.wait() dans un try/except pour éviter
    que le programme ne s'arrête en cas d'erreur de PortAudio.
    """
    try:
        sd.play(signal, samplerate)
        sd.wait()
    except Exception as e:
        print("Erreur lors de la lecture audio :", e)

###############################################################################
#                 FONCTION DE VALIDATION DES FANIONS
###############################################################################
def is_valid_fanion(received, expected, tolerance=3):
    """
    Compare deux listes de bits.
    Retourne True si le nombre de bits différents est inférieur ou égal à 'tolerance'.
    (Tolérance augmentée à 3 pour SNR=15 dB.)
    """
    mismatches = sum(1 for r, e in zip(received, expected) if r != e)
    return mismatches <= tolerance

###############################################################################
#                        FONCTIONS POUR L'ÉMISSION
###############################################################################
def crcemission(bits):
    """
    Calcule et ajoute 4 bits de CRC pour chaque octet (8 bits).
    Diviseur = '1001' (9 décimal).
    """
    diviseur = int("1001", 2)
    resultat_crc = []
    local_bits = bits[:]  # copie pour ne pas modifier la liste originale

    while len(local_bits) >= 8:
        donne_str = ''.join(map(str, local_bits[:8]))  # 8 bits
        local_bits = local_bits[8:]
        # Ajout "0000"
        div_str = donne_str + "0000"
        div_int = int(div_str, 2)
        _, reste = divmod(div_int, diviseur)
        # On stocke l'octet + 4 bits de CRC
        resultat_crc.extend(map(int, donne_str))
        resultat_crc.extend(map(int, format(reste, '04b')))
    print("=== ÉMISSION : CRC ===")
    print("Données + CRC :", resultat_crc)
    return resultat_crc

def trame_emission(bits, type_fichier="txt"):
    """
    Construit la trame :
      - fanion début = [1]*8
      - octet protocole (ex: [1,1,0,0,0,0,1,1] pour "txt")
      - bits + CRC
      - fanion fin = [0]*8
    """
    fanion_debut = [1]*8
    fanion_fin   = [0]*8
    prot_txt     = [1,1,0,0,0,0,1,1] if type_fichier=="txt" else [0,0,0,0,0,0,0,0]
    trame = fanion_debut + prot_txt + bits + fanion_fin
    print("=== ÉMISSION : TRAME ===")
    print("Fanion début :", fanion_debut)
    print("Fanion fin   :", fanion_fin)
    print("Protocole    :", prot_txt)
    print("Trame finale :", trame)
    return trame

def Manchester_encode(bits):
    """
    Codage Manchester : 1 -> (1,0), 0 -> (0,1)
    """
    code = []
    for b in bits:
        if b == 1:
            code.extend([1,0])
        else:
            code.extend([0,1])
    print("=== ÉMISSION : Manchester ===")
    print("Manchester (extrait) :", code[:20], "...")
    return code

def modulation_ASK_FSK(bits_manchester, prefix="Message"):
    """
    Modulation ASK & FSK, enregistrement dans 2 CSV : {prefix}_ASK.csv, {prefix}_FSK.csv.
    Lecture audio (utilise safe_play) et affichage graphique (zoom sur 2000 échantillons).
    """
    bits_dupliques = np.repeat(bits_manchester, Ns)
    N = len(bits_dupliques)
    t = np.arange(N) / Fe

    # ASK
    porteuse = np.sin(2 * np.pi * Fp_ASK * t)
    ASK = bits_dupliques * porteuse

    # FSK
    s1 = np.sin(2 * np.pi * fp1 * t)
    s2 = np.sin(2 * np.pi * fp2 * t)
    FSK = np.where(bits_dupliques == 1, s1, s2)

    # Sauvegarde
    ask_file = f"{prefix}_ASK.csv"
    fsk_file = f"{prefix}_FSK.csv"
    np.savetxt(ask_file, ASK, fmt="%.6f", delimiter=',')
    np.savetxt(fsk_file, FSK, fmt="%.6f", delimiter=',')
    print(f"=== ÉMISSION : Modulation => fichiers {ask_file}, {fsk_file} ===")

    # Lecture audio avec safe_play
    safe_play(ASK, Fe)
    safe_play(FSK, Fe)

    # Graphiques
    plt.figure(figsize=(10,4))
    plt.plot(t[:2000], bits_dupliques[:2000], 'b', label='Bits dupliqués')
    plt.title(f"{prefix} - Bits Manchester (zoom 2000)")
    plt.legend()
    plt.grid(True)

    plt.figure(figsize=(10,4))
    plt.plot(t[:2000], ASK[:2000], 'r', label='Signal ASK')
    plt.title(f"{prefix} - ASK (zoom 2000)")
    plt.legend()
    plt.grid(True)

    plt.figure(figsize=(10,4))
    plt.plot(t[:2000], FSK[:2000], 'g', label='Signal FSK')
    plt.title(f"{prefix} - FSK (zoom 2000)")
    plt.legend()
    plt.grid(True)

    plt.show()

###############################################################################
#                     FONCTIONS POUR LA RÉCEPTION
###############################################################################
def add_noise_to_signal(signal, snr_db=20):
    """
    Ajoute un bruit gaussien blanc en fonction du SNR (en dB).
    snr_db = 20 => bruit modéré, 30 => bruit faible, 10 => bruit fort.
    """
    snr_linear = 10 ** (snr_db / 10.0)
    signal_power = np.mean(signal ** 2)
    noise_power  = signal_power / snr_linear
    noise = np.sqrt(noise_power) * np.random.normal(size=len(signal))
    return signal + noise

def demodulation_ASK_FSK(prefix="Message", add_noise=False, snr_db=20):
    """
    Lit les fichiers {prefix}_ASK.csv et {prefix}_FSK.csv,
    applique (ou non) du bruit, puis démodule la voie ASK.
    Retourne la séquence binaire (liste de 0/1).
    Utilise un seuil de décision fixé à 0.0 pour compenser le bruit.
    """
    ask_file = f"{prefix}_ASK.csv"
    fsk_file = f"{prefix}_FSK.csv"
    ASK_signal = np.genfromtxt(ask_file, delimiter=',')
    FSK_signal = np.genfromtxt(fsk_file, delimiter=',')

    if add_noise:
        print(f"--- RÉCEPTION : On ajoute du bruit SNR={snr_db} dB sur {prefix}. ---")
        ASK_signal = add_noise_to_signal(ASK_signal, snr_db)
        FSK_signal = add_noise_to_signal(FSK_signal, snr_db)
    else:
        print(f"--- RÉCEPTION : Pas de bruit pour {prefix}. ---")

    N = len(ASK_signal)
    t = np.arange(N) / Fe

    # Démodulation ASK
    porteuse = np.sin(2 * np.pi * Fp_ASK * t)
    produit  = ASK_signal * porteuse

    bits = []
    threshold = 0.0  # Seuil fixé à 0.0 pour compenser le bruit
    for i in range(0, N, Ns):
        portion = produit[i : i + Ns]
        val = np.trapz(portion)
        bits.append(1 if val > threshold else 0)
    return bits

def Manchester_decode(bits):
    """
    Décodage Manchester : (1,0)->1, (0,1)->0.
    """
    decode = []
    for i in range(0, len(bits), 2):
        b1 = bits[i]
        b2 = bits[i+1]
        if (b1, b2) == (1, 0):
            decode.append(1)
        elif (b1, b2) == (0, 1):
            decode.append(0)
        # Sinon, motif inconnu (on l'ignore ici)
    return decode

def trame_reception(bits):
    """
    Extrait fanion début, fanion fin, octet protocole et data+CRC.
    Utilise une tolérance de 3 bits pour la vérification des fanions.
    Retourne (data_crc, type_fichier) ou (None, None) en cas d'erreur.
    """
    if len(bits) < 16:
        return None, None
    fanion_debut = bits[:8]
    fanion_fin   = bits[-8:]
    core = bits[8:-8]

    if not (is_valid_fanion(fanion_debut, [1]*8, tolerance=3) and is_valid_fanion(fanion_fin, [0]*8, tolerance=3)):
        return None, None
    if len(core) < 8:
        return None, None

    protocole = core[:8]
    data_crc  = core[8:]
    if protocole == [1,1,0,0,0,0,1,1]:
        type_fic = "txt"
    else:
        type_fic = "?"
    return data_crc, type_fic

def crcreception(bits):
    """
    Lit par blocs de 12 bits (8 data + 4 CRC).
    Diviseur '1001' = 9 décimal.
    Retourne la liste de bits data si OK, sinon None.
    """
    diviseur = int("1001", 2)
    idx = 0
    data_res = []
    integrite_ok = True

    while idx + 12 <= len(bits):
        bloc = bits[idx:idx+12]
        idx += 12
        data_8  = bloc[:8]
        reste_4 = bloc[8:]
        data_str = "".join(str(x) for x in data_8)
        data_plus_0000 = data_str + "0000"
        data_dec = int(data_plus_0000, 2)
        reste_calc = data_dec % diviseur
        reste_calc_str = format(reste_calc, '04b')
        reste_str = "".join(str(x) for x in reste_4)
        if reste_calc_str != reste_str:
            integrite_ok = False
        data_res.extend(data_8)

    if integrite_ok:
        return data_res
    else:
        return None

def ASCII_decode(bits, type_fic):
    """
    Convertit 8 bits -> 1 caractère ASCII.
    Retourne la chaîne ASCII.
    """
    if type_fic != "txt" or not bits:
        return "[Protocole inconnu / bits vides]"
    chaine = ""
    for i in range(0, len(bits), 8):
        octet = bits[i:i+8]
        if len(octet) < 8:
            break
        val = int("".join(str(b) for b in octet), 2)
        chaine += chr(val)
    return chaine

###############################################################################
#             HALF-DUPLEX + BRUIT + SEUIL ADAPTÉ
###############################################################################
def emission_reception(prefix, message, add_noise=False, snr_db=20):
    """
    Émet 'message' (appliquant CRC, trame, Manchester, modulation ASK/FSK)
    et enregistre les fichiers {prefix}_ASK.csv et {prefix}_FSK.csv.
    Puis lit ces fichiers (optionnellement bruités), démodule et reconstruit le message.
    Retourne le message ASCII final si OK, sinon None.
    """
    # Conversion ASCII -> bits
    bits_texte = []
    for c in message:
        bits_texte.extend(int(x) for x in format(ord(c), '08b'))
    
    # CRC
    bits_crc = crcemission(bits_texte)
    # Trame
    bits_trame = trame_emission(bits_crc, "txt")
    # Manchester encode
    bits_manch = Manchester_encode(bits_trame)
    # Modulation
    modulation_ASK_FSK(bits_manch, prefix=prefix)
    
    print(f"\n=== RÉCEPTION DE {prefix} ===")
    # Démodulation
    ask_bits = demodulation_ASK_FSK(prefix=prefix, add_noise=add_noise, snr_db=snr_db)
    # Manchester decode
    decode_manch = Manchester_decode(ask_bits)
    # Extraction de la trame
    data_crc, type_fic = trame_reception(decode_manch)
    if data_crc is None or type_fic is None:
        print(f"Trame invalide pour {prefix}.\n")
        return None
    # Vérification CRC
    data_sans_crc = crcreception(data_crc)
    if data_sans_crc is None:
        print(f"CRC invalide pour {prefix}.\n")
        return None
    # ASCII decode
    final_txt = ASCII_decode(data_sans_crc, type_fic)
    print(f"Message final reçu pour {prefix} :", final_txt, "\n")
    return final_txt

def half_duplex_demo(add_noise=False, snr_db=20):
    """
    1) L'utilisateur saisit un message ("Message").
    2) On émet et reçoit ce message (avec ou sans bruit).
    3) Si tout va bien, on émet un ACK ("OK").
    4) On reçoit l'ACK.
    """
    # Phase 1 : Saisie du message
    print("=== PHASE 1 : Saisissez votre message ===")
    texte = ""
    while not texte:
        texte = input("Votre message : ")
    
    print("\n=== ÉMISSION & RÉCEPTION DU MESSAGE ===")
    msg_recu = emission_reception("Message", texte, add_noise=add_noise, snr_db=snr_db)
    if msg_recu is None:
        print("Échec de réception du message, pas d'ACK.\n")
        return
    
    # Phase 2 : Émission et réception d'un ACK
    print("=== PHASE 2 : ÉMISSION & RÉCEPTION D'UN ACK ===")
    ack_str = "OK"
    ack_recu = emission_reception("ACK", ack_str, add_noise=add_noise, snr_db=snr_db)
    if ack_recu is None:
        print("ACK non reçu ou corrompu.\n")
        return
    print("=== FIN DU HALF-DUPLEX ===")

###############################################################################
#                          PROGRAMME PRINCIPAL
###############################################################################
if __name__=="__main__":
    print("""
==================================================
  DEMO : MODE HALF-DUPLEX + CRC + BRUIT (OPTION)
==================================================
Choisissez si vous voulez ajouter du bruit (True/False) 
et le niveau de SNR (ex. 20 dB)
""")
    # Paramètres de bruit
    noise_flag = True
    noise_snr  = 15  # SNR = 15 dB (bruit modéré)
    
    half_duplex_demo(add_noise=noise_flag, snr_db=noise_snr)



  DEMO : MODE HALF-DUPLEX + CRC + BRUIT (OPTION)
Choisissez si vous voulez ajouter du bruit (True/False) 
et le niveau de SNR (ex. 20 dB)

=== PHASE 1 : Saisissez votre message ===



=== ÉMISSION & RÉCEPTION DU MESSAGE ===
=== ÉMISSION : CRC ===
Données + CRC : [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0]
=== ÉMISSION : TRAME ===
Fanion début : [1, 1, 1, 1, 1, 1, 1, 1]
Fanion fin   : [0, 0, 0, 0, 0, 0, 0, 0]
Protocole    : [1, 1, 0, 0, 0, 0, 1, 1]
Trame finale : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
=== ÉMISSION : Manchester ===
Manchester (extrait) : [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0] ...
=== ÉMISSION : Modulation => fichiers Message_ASK.csv, Message_FSK.csv ===
||PaMacCore (AUHAL)|| Warning on line 521: err=''!obj'', msg=

PortAudioError: Error opening OutputStream: Internal PortAudio error [PaErrorCode -9986]